In [1]:
import csv
import numpy as np
import pandas as pd
from dfs_semantics import MeaningSpace, Semantics
import dfs_semantics as dfs

In [2]:
p = ['happy(john)', 'walks(john)', 'happy(mary)', 'walks(mary)']

In [30]:
s = Semantics(propositions=p)
print(s.space)
print(s.space.matrix)

{'happy(john)', 'happy(mary)', 'walks(john)', 'walks(mary)'}
[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 1 1 1]]


In [31]:
john = s.getWordSemantics('john')
mary = s.getWordSemantics('mary')
walks = s.getWordSemantics('walks')
s.setMerge(walks, john).T

array([[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]])

In [42]:
y = s.setMerge(john, walks)
s.space.printSet(dfs.matrix_to_set(y))
x =s.setMerge(mary, walks)
s.space.printSet(dfs.matrix_to_set(x))
print(y.T)
print(x.T)

{'walks(john)'}
{'walks(mary)'}
[[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]]
[[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]]


In [40]:
#walks(john) & walks(mary) (make this code better)
walks_johnandmary = dfs.vector_and(y, x).astype(int)
walks_johnandmary.T

array([[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]])

In [34]:
dfs.vector_and(s.setMerge(walks, john), s.setMerge(walks, mary)).astype(int).T

array([[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]])

In [28]:
# walks(john&mary)
john_and_mary = s.setAnd(mary, john).astype(int)
merged = s.setMerge(walks, john_and_mary)
merged

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 1, 1]])

In [27]:
assert (merged[:, 1] == walks_johnandmary).all()
print(True)

True


In [29]:
john_and_mary

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 1, 1]])

In [10]:
a = np.array([[1,0,0], [1,0,1]])
b = np.array([[1,1,0], [1,1,0]])
c = dfs_semantics.matrix_imp(a, b)
c

AttributeError: module 'dfs_semantics' has no attribute 'matrix_imp'

In [6]:
c[:, np.all(True)]

array([[[ True,  True,  True]],

       [[ True,  True, False]]])

In [12]:
s.merge(a, b)

array([[1, 0],
       [1, 0]])

In [8]:
file = 'model.observations'
# niet geheel triviaal hoe deze datastructuren zouden moeten werken, dus ik probeer gewoon wat...
model_df = pd.read_csv(file, sep=' ', header=0)
model_matrix = np.array(model_df)
propositions = list(model_df.columns)
prop_dict = {tuple(np.array(model_df.iloc[:, index])) : prop for index, prop in enumerate(propositions)}

In [9]:
def matrix_to_set(Matrix):
    return set(map(tuple, Matrix))

In [16]:
from itertools import product
print(list(product([0,1], repeat=4)))
print(len(list(product([0,1], repeat=4))))

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)]
16


In [4]:
def set_to_matrix(Set):
    array = np.array([elem for elem in Set])
    return array

In [5]:
def print_set(Set, prop_dict, raw=False):
    if not raw:
        try:
            print(set([prop_dict[elem] for elem in Set]))
        except KeyError:
            print(set([f'v_{i}' for i, elem in enumerate(Set)]))
    else:
        print(set([elem for elem in Set]))

In [6]:
def interpretation(semantics_rep):
    return np.mean(set_to_matrix(semantics_rep), axis=0)

$[[w]]^M = \{V_n(p)\ | w \in p\}$ for all $p \in \mathcal{P}$

## Negatie ##

$[[\neg w]]^M = \{\neg V_n(p)\ | w \in p\}$ for all $p \in \mathcal{P}$ 

OF 

$[[\neg w]]^M  = \{\mathcal{P} \setminus [[w]]^M\}$

## Conjunctie ##

$[[w_1 \land w_2]]^M = \{[[w_1]]^M \cap [[w_2]]^M\}$

OF

$[[w_1 \land w_2]]^M = v_1(w_1) \wedge \cdots \wedge v_n(w_1) \land v_1(w_2) \wedge \cdots \wedge v_n(w_2)$

OF ?

$[[w_1 \land w_2]]^M = \{M_i | (w_1, w_2) \in \mathcal{M} \} $

## Disjunctie

$[[w_1 \lor w_2]]^M = \{[[w_1]]^M \cup [[w_2]]^M\}$

In [100]:
def getSemantics(word, matrix, propositons, output='set'):
    relevantProps = [index for index, prop in enumerate(propositions) if word in prop]
    vectors = matrix[:, relevantProps]
    return matrix_to_set(vectors.T) if output == 'set' else vectors

In [122]:
def negate_elem(semantics_rep): #works only on matrices
    if not isinstance(semantics_rep, np.ndarray):
        semantics_rep = set_to_matrix(semantics_rep)
    #coole truc: gebruik bitwise XOR operator  (^) met 1 0^1 = 1, 1^1 = 1
    #geen gezeik met converteren naar truth values
    return matrix_to_set(semantics_rep^1)

def negate(semantics_rep):
    if not isinstance(semantics_rep, set):
        semantics_rep = matrix_to_set(semantics_rep)
    world = matrix_to_set(model_matrix.T)
    print_set(world, prop_dict)
    return world.difference(semantics_rep)

In [127]:
def AND(p1, p2): #works only on sets
    return p1.intersection(p2)

def AND_elem(p1, p2): #works only on matrices
    if not isinstance(p1, np.ndarray) or not isinstance(p2, np.ndarray):
        p1 = set_to_matrix(p1)
        p2 = set_to_matrix(p2)
    return matrix_to_set(p1*p2)

In [107]:
def OR(p1, p2): #works only on sets
    return p1.union(p2)

### Demo tot nu toe ###
Let op dat de negatie zoals geimplementeerd als enige operatie (tot nu toe) niet-atomische propositierepresentaties oplevert. Dat betekent dat elke semantiek van arbitraire complexiteit niet-atomische representaties op levert als er ook maar 1 negatie ergens zit. Hierdoor werken de definites van AND en OR niet meer: AND(p1, negate_flip(p2)) is altijd de empty set! Daarom heb ik een tweede negatie operatie gedefinieerd.

In [118]:
sleep = getSemantics('sleep', model_matrix, propositions)
nick = getSemantics('nick', model_matrix, propositions)
zelda = getSemantics('zelda', model_matrix, propositions)
tease = getSemantics('tease', model_matrix, propositions)


print('[[sleep]]', ':')
print_set(sleep, prop_dict)
print('\n')
print('[[nick]]', ':')
print_set(nick, prop_dict)
print('\n')
print('[[not nick]]', ':')
print_set(negate(nick), prop_dict)
print('\n')
print('[[zelda]]', ':')
print_set(zelda, prop_dict)
print('\n')
print('[[tease]]', ':')
print_set(tease, prop_dict)
print('\n')
print('[[sleep ^ nick]]', ':')
print_set(AND(sleep,nick), prop_dict)
print('\n')
print('[[tease ^ nick]]', ':')
print_set(AND(tease,nick), prop_dict)
print('\n')
print('[[nick V zelda]]', ':')
print_set(OR(nick, nick), prop_dict)

[[sleep]] :
{'sleep(zelda)', 'sleep(nick)', 'sleep(jess)', 'sleep(winston)'}


[[nick]] :
{'tease(nick,winston)', 'tease(nick,zelda)', 'tease(jess,nick)', 'tease(nick,nick)', 'sleep(nick)', 'read(nick)', 'tease(zelda,nick)', 'tease(winston,nick)', 'tease(nick,jess)'}


[[not nick]] :
{'tease(zelda,zelda)', 'tease(zelda,nick)', 'read(winston)', 'tease(winston,winston)', 'tease(jess,zelda)', 'tease(winston,jess)', 'read(zelda)', 'sleep(zelda)', 'tease(nick,zelda)', 'tease(nick,jess)', 'tease(nick,winston)', 'tease(zelda,jess)', 'tease(jess,nick)', 'tease(winston,zelda)', 'read(nick)', 'sleep(winston)', 'read(jess)', 'tease(jess,winston)', 'sleep(jess)', 'tease(zelda,winston)', 'tease(nick,nick)', 'sleep(nick)', 'tease(winston,nick)', 'tease(jess,jess)'}
{'tease(zelda,winston)', 'tease(zelda,jess)', 'tease(winston,jess)', 'tease(winston,zelda)', 'tease(jess,winston)', 'tease(zelda,zelda)', 'read(zelda)', 'tease(jess,jess)', 'sleep(winston)', 'sleep(zelda)', 'read(jess)', 'read(winston)', 

### Implicatie ###

Eerste idee: Gebruik $A \rightarrow B = \neg(A \land \neg B)$

In [115]:
def imply(p1, p2):
    return negate(AND(p1, negate(p2)))

In [119]:
print('[[nick imply zelda]]', ':')
print_set(imply(nick, zelda), prop_dict)

[[nick imply zelda]] :
{'tease(zelda,zelda)', 'tease(zelda,nick)', 'read(winston)', 'tease(winston,winston)', 'tease(jess,zelda)', 'tease(winston,jess)', 'read(zelda)', 'sleep(zelda)', 'tease(nick,zelda)', 'tease(nick,jess)', 'tease(nick,winston)', 'tease(zelda,jess)', 'tease(jess,nick)', 'tease(winston,zelda)', 'read(nick)', 'sleep(winston)', 'read(jess)', 'tease(jess,winston)', 'sleep(jess)', 'tease(zelda,winston)', 'tease(nick,nick)', 'sleep(nick)', 'tease(winston,nick)', 'tease(jess,jess)'}
{'tease(zelda,zelda)', 'tease(zelda,nick)', 'read(winston)', 'tease(winston,winston)', 'tease(jess,zelda)', 'tease(winston,jess)', 'read(zelda)', 'sleep(zelda)', 'tease(nick,zelda)', 'tease(nick,jess)', 'tease(nick,winston)', 'tease(zelda,jess)', 'tease(jess,nick)', 'tease(winston,zelda)', 'read(nick)', 'sleep(winston)', 'read(jess)', 'tease(jess,winston)', 'sleep(jess)', 'tease(zelda,winston)', 'tease(nick,nick)', 'sleep(nick)', 'tease(winston,nick)', 'tease(jess,jess)'}
{'tease(zelda,winston)'

In [135]:
print_set(AND_elem(nick, zelda), prop_dict)

{'v_3', 'v_7', 'v_1', 'v_5', 'v_8', 'v_2', 'v_0', 'v_6', 'v_4'}


In [137]:
np.array([1, 0, 1]) * np.array([0, 1, 1]) * np.array([1, 1, 1])

array([0, 0, 1])